In [65]:
import codecs
import time
import re
import pymongo
from pyspark.mllib.feature import HashingTF, IDF
from pymongo import MongoClient 
import findspark
import re
#from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
findspark.init()
import pyspark
#sc = pyspark.SparkContext(appName="myAppName")#myAppName
from __future__ import print_function
from pyspark import SparkContext
#reference  https://www.zybuluo.com/jewes/note/35032

### import dictionary

In [48]:
jieba.set_dictionary('/home/cloudera/Desktop/project/dics/dict.txt.big.txt')   #切換至中文繁體字庫
jieba.load_userdict('/home/cloudera/Desktop/project/dics/dict_keyw.txt')       #加入自建詞庫
jieba.load_userdict('/home/cloudera/Desktop/project/dics/dict_keyw_new.txt')

Building prefix dict from /home/cloudera/Desktop/project/dics/dict.txt.big.txt ...
Loading model from cache /tmp/jieba.ubd40686f1bbfd4d132e1423888a99ba6.cache
Loading model cost 0.903 seconds.
Prefix dict has been built succesfully.


### get article has done jieba  from mongoDB

In [104]:
#tag1 = "蔡英文"
tag2 = "_"
#預設就是自己
client = MongoClient('10.120.28.6',27017)
database = client['test_new1']
collection =database['test1']

querry_resp = collection.find({"$and":[
           # {"content":{"$regex":tag1}},
            {"content":{"$regex":tag2}},
        ]})
#querry_resp = collection.find({
#        "date":{"$regex":"201601"}
#    })
print (querry_resp.count())
#print "查詢結果的第一筆資料標題:",querry_resp[0]["title"]
#print "找到都少筆數:", querry_resp.count()
with codecs.open("/home/cloudera/Desktop/project/practice/project_TFIDF.json",'w',encoding='utf-8') as f:
    for post in querry_resp: 
        writer=[]
        writer =  str(post['_id']) + "|"+post['content']
        #writer = post['content']
        #print  type(summary)
        #writer=('/'.join(jieba.cut(summary)))  #把一篇文章切完詞後 再用 "/" 接起來放在一起!
        writer1=writer+'\n'
        f.write(writer1) 

6217


### wordcount (jieba)   --  one articles   +  textfile  --

In [6]:
documents = sc.textFile("file:/home/cloudera/Desktop/project/20160428.json")
wordRDD = documents.flatMap(lambda x: jieba.cut(x))
wordFreRDD = wordRDD.map(lambda x: (x, 1))
#print (wordFreRDD.collect())
counts = wordFreRDD.reduceByKey(lambda x ,y : x+y)
#print ((counts.collect()))
list_words=counts.collect()
lis_sorted = sorted(list_words,key=lambda li:li[1], reverse=True)
print (type(lis_sorted))
for ele in lis_sorted:
    print (ele[0],ele[1])   # show keywords and count

<type 'list'>
_ 56
的 10
義大 7
「 6
觀眾 6
跨年 6
」 6
、 6
也 5
晚會 5
年 4
在 4
不 4
衣服 3
煙火 3
》 3
3 3
世界 3
《 3
與 3
由 3
點 2
謝金燕 2
觀光 2
昨晚 2
表演 2
2014 2
現場 2
迎接 2
高雄 2
達 2
分鐘 2
廣告 2
浪漫 2
姐姐 2
是 2
胡瓜 2
小姐 2
施放 2
義聯 2
蕭亞軒 2
情緒 2
讓 2
呈現 2
6 2
主持人 2
感謝 2
登場 2
暖 2
今年 2
瀑布 2
為 2
全台 2
集團 2
紫耀義大 2
跨 2
主持 2
帶動 2
天心 2
要 1
去年 1
特別 1
歌手 1
銀冠動態 1
40 1
原汁原味 1
天氣 1
多 1
貢獻 1
  1
發展 1
20 1
機智 1
遇到 1
溫 1
想到 1
尬 1
到 1
轉播 1
最強 1
開場 1
組曲 1
最後由 1
天空 1
再次 1
並特別 1
套服 1
快歌 1
象徵 1
出動 1
進行 1
祝福 1
上台 1
湧入 1
半小時 1
輪番 1
熱度 1
問答 1
火團隊 1
豪 1
四套 1
昇 1
歐漢聲 1
由國內 1
和 1
金冠 1
外煙 1
家家 1
能脫 1
3D 1
非常 1
團體 1
並帶 1
評選為 1
結束 1
氣氛 1
520 1
南台 1
白安 1
人潮 1
做出 1
鑽石 1
等 1
菊 1
倒數 1
兩場 1
自己 1
昨準備 1
打 1
煙火秀 1
邀民眾 1
無價 1
力量 1
發抖 1
因 1
金燕 1
顫 1
30 1
扮瀟灑 1
露出 1
驚呼 1
自掏腰包 1
陳 1
秒 1
她 1
繼續 1
舉辦 1
各換 1
最 1
超過 1
量身 1
說話 1
演唱 1
60 1
得 1
市府 1
包 1
朱俐靜 1
打造 1
炒熱 1
連連 1
舞台 1
第 1
魔幻 1
地 1
火花 1
2 1
冷 1
認為 1
萬人 1
個 1
連趕 1
4 1
日本 1
時 1
合作 1
以帥 1
軍 1
保持 1
整個 1
老手 1
V 1
上 1
氣深 1
做 1
你 1
西裝 1
15 1
大主題 1
新年 1
陪 1
花媽 1
將 1
自行 1
萬發 1
充滿 1
沒 1
興奮 1
一起 1
更策畫 1
道題 1
裝是 1
設計近

### wordcount (jieba) --  many articles  +    connect mongoDB --

In [105]:
# many articles 
from pymongo import MongoClient 
#預設就是自己
client = MongoClient('10.120.28.6',27017)
database = client['testPyspark']
collection =database['news']

many_to_count=[]
i=0
list_news=[]
content_after_jieba=[]
for post in collection.find():
    i=i+1
    list_news=[]
    list_news.append(post['content'])
    print (post['content'])
    documents=sc.parallelize(list_news)
    wordRDD = documents.flatMap(lambda x: jieba.cut(x))                                                                                                                                                                                                                                                                               
    print ((wordRDD.collect()))
    wordFreRDD = wordRDD.map(lambda x: (x, 1))
    counts = wordFreRDD.reduceByKey(lambda x ,y : x+y)
    #print (counts.collect())    #OK
    count_all=counts.collect()
    #print (type(count_all))
    lis_sorted = sorted(count_all,key=lambda li:li[1], reverse=True)
    #print (lis_sorted)
    for ele in lis_sorted:
        print (ele[0],ele[1])   # show keywords and count

    print ("---------next one----------")

總統當選人蔡英文昨日南下與台達電創辦人鄭崇華等人對談_由於選前雙方曾就電價問題隔空交鋒_蔡昨天不僅主動澄清電價與能源政策_並強調之所以說出「10年電價不會大幅上漲」的原因有3_包括落實節能讓成本下降、綠能科技規模擴大可降低成本_以及修改《電業法》讓廠、網分離等_小戶不漲大戶漲蔡英文選後展開產業參訪之旅_昨天鎖定綠能產業_南下參訪台達電且舉行座談_蔡在致詞時主動回應鄭曾質疑的電價問題_「我知道鄭創辦人很關心電價的問題」_因為這牽涉整體能源政策的規劃_而台灣必須制定一套合理且永續的能源政策_「相信創辦人跟我都有一樣的共識」_蔡也澄清_過去她的電價政策_被簡化成10年不漲電價的既定印象_但民進黨其實是有完整規劃與思考的_她的電價主張_是以基本度數不漲價為原則_減少電價波動_因為電價一直是台灣生產、生活成本的最重要的事情之一；未來即便要調整_也會考量物價跟一般人負擔能力_在民生用電小戶部分_「現階段沒有調漲計畫」_但用電大戶則是重點_未來將鼓勵大戶配合節電措施_以及依照「時間電價」和「需量競價」來調度電力_企業政府3管齊下蔡英文也說_她之所以有把握地說「10年內電價不會大幅上漲」_原因有3_其一_只要落實節能_電力使用效率增加_就不需要多繳電費；其二_綠能現階段價格雖高_但科技進步、規模擴大後_成本自然下降；其三_新政府將會修改電業法_推動廠、網分離_提升台電經營效率_讓電價更透明、更為合理_她也強調_新政府會持續跟產業界溝通_視總體經濟情況_做適度的檢討_並且作調整及改善_蔡英文並盛讚台達電_表示她的產業之旅在「新能源」、「環保節能產業」這一站特別來到台達電_是因為「節能」是最快解決能源供給問題的方法_而台達電可以說是節能與再生能源相關產品最齊全的公司_她也開玩笑說_「我的講稿裡原本是寫『之一』啦_我去掉了！」讓在場台達電主管們一陣鼓掌爆笑_省電標竿台達電蔡英文說_台灣要建立一個更乾淨、更有效率的能源系統_一定要借助台達電的發展經驗_也期待台達電能夠整合不同事業群產品_規劃更好的解決方案來面對全球的市場_鄭崇華也當場允諾_未來一定會和新政府合作_推動綠能產業_(中國時報)
[u'\u7e3d\u7d71', u'\u7576\u9078\u4eba', u'\u8521', u'\u82f1\u6587', u'\u6628\u65e5', u'\u5357\u4e0b'

### TF-IDF --case1

In [106]:
##count features
documents = sc.textFile("file:/home/cloudera/Desktop/project/practice/project_TFIDF.json")
collection_count=documents.count()

documents_count_feature=documents.map(lambda line: (line.split("|")[1]) ).flatMap(lambda x: jieba.cut(x)).map(lambda x :(x,1))
documents_count_feature1=documents_count_feature.reduceByKey(lambda x,y : x+y)
count_features=documents_count_feature1.count()
print (count_features)

169995


In [107]:
documents = sc.textFile("file:/home/cloudera/Desktop/project/practice/project_TFIDF.json")
collection_count=documents.count()
#print (collection_count)

print  ("查詢結果資料總筆數:" ,collection_count)


time_start_jieba = time.clock()
documents1=documents.map(lambda line: (line.split("|")[0],line.split("|")[1])).flatMapValues(lambda x: jieba.cut(x)).groupByKey().mapValues(list).map(lambda x: ( x[1]))
#print (documents1.collect())
time_end_jieba = time.clock()

#print (documents1.collect())

time_for_jieba= time_end_jieba-time_start_jieba

print ("I/O time + jieba的時間:" ,time_for_jieba,"seconds")
print ("特徵值數量:" , count_features)

#print (documents.collect())
#wordRDD1 = documents.flatMapValues(lambda x: jieba.cut(x))#.map(lambda x: ((x[0], x[1]), 1)).reduceByKey(add)#.groupByKey().mapValues(list)     #append('/'.join(jieba.cut(summary)))                                                                                                                                                                                                                                                                           
#wordRDD2 =wordRDD1.groupByKey().mapValues(list)
#wordRDD3=wordRDD2.map(lambda x: ( x[1]))
#print (wordRDD2.collect())

time_start_TFIDF = time.clock()
hashingTF = HashingTF()
tf = hashingTF.transform(documents1)
tf.cache()
#print ("--------------------------")
#print ("tf:")
#print (tf.collect()) 
idf = IDF().fit(tf)
tfidf = idf.transform(tf)   #<class 'pyspark.mllib.feature.IDFModel'>
time_end_TFIDF= time.clock()

time_for_TFIDF= time_end_TFIDF-time_start_TFIDF
print ("計算tf-idf的時間" , time_for_TFIDF,"seconds")

#print ("--------------------------")
#print ("tfidf:")
#print (tfidf.collect()) 

#combined = documents.zip(tfidf)
#print ("--------------------------")
#print (combined.collect()) 


查詢結果資料總筆數: 6217
I/O time + jieba的時間: 0.0 seconds
特徵值數量: 169995
計算tf-idf的時間 0.05 seconds


### sorted  DIY

In [ ]:
#print (combined.collect()[0][1]) 
keywords=(combined.collect()[0][0])
#SparseVector=(combined.collect()[0]) 
#print ((SparseVector))
#print (type(tfidf.collect()[0]))
for word in keywords:
    print (word)

#for word, Vector in zip(keywords, SparseVector):
    #print (word, Vector)

In [56]:
keywords=[u'\u7e3d\u7d71', u'\u7576\u9078\u4eba', u'\u8521', u'\u82f1\u6587', u'\u6628\u65e5', u'\u5357\u4e0b', u'\u8207', u'\u53f0\u9054', u'\u96fb\u5275', u'\u8fa6\u4eba', u'\u912d\u5d07\u83ef', u'\u7b49', u'\u4eba', u'\u5c0d', u'\u8ac7', u'_', u'\u7531', u'\u65bc', u'\u9078\u524d', u'\u96d9\u65b9', u'\u66fe', u'\u5c31', u'\u96fb\u50f9', u'\u554f\u984c', u'\u9694', u'\u7a7a\u4ea4\u92d2', u'_', u'\u8521', u'\u6628\u5929', u'\u4e0d\u50c5', u'\u4e3b\u52d5', u'\u6f84\u6e05', u'\u96fb\u50f9', u'\u8207', u'\u80fd\u6e90', u'\u653f\u7b56', u'_', u'\u4e26\u5f37', u'\u8abf', u'\u4e4b\u6240\u4ee5', u'\u8aaa\u51fa', u'\u300c', u'10', u'\u5e74', u'\u96fb\u50f9', u'\u4e0d\u6703', u'\u5927\u5e45', u'\u4e0a\u6f32', u'\u300d', u'\u7684', u'\u539f\u56e0', u'\u6709', u'3', u'_', u'\u5305\u62ec', u'\u843d\u5be6\u7bc0', u'\u80fd', u'\u8b93', u'\u6210\u672c', u'\u4e0b\u964d', u'\u3001', u'\u7da0\u80fd', u'\u79d1\u6280', u'\u898f\u6a21', u'\u64f4\u5927', u'\u53ef', u'\u964d\u4f4e\u6210\u672c', u'_', u'\u4ee5\u53ca', u'\u4fee\u6539', u'\u300a', u'\u96fb\u696d\u6cd5', u'\u300b', u'\u8b93', u'\u5ee0', u'\u3001', u'\u7db2\u5206', u'\u96e2', u'\u7b49', u'_', u'\u5c0f\u6236', u'\u4e0d\u6f32', u'\u5927\u6236', u'\u6f32', u'\u8521', u'\u82f1\u6587', u'\u9078\u5f8c\u5c55', u'\u958b\u7522\u696d', u'\u53c3\u8a2a', u'\u4e4b', u'\u65c5', u'_', u'\u6628\u5929', u'\u9396\u5b9a', u'\u7da0\u80fd', u'\u7522\u696d', u'_', u'\u5357\u4e0b', u'\u53c3\u8a2a', u'\u53f0\u9054', u'\u96fb\u4e14', u'\u8209\u884c', u'\u5ea7', u'\u8ac7', u'_', u'\u8521\u5728\u81f4', u'\u8a5e\u6642', u'\u4e3b\u52d5', u'\u56de\u61c9', u'\u912d\u66fe', u'\u8cea\u7591', u'\u7684', u'\u96fb\u50f9', u'\u554f\u984c', u'_', u'\u300c', u'\u6211', u'\u77e5\u9053', u'\u912d\u5275', u'\u8fa6\u4eba', u'\u5f88', u'\u95dc\u5fc3', u'\u96fb\u50f9', u'\u7684', u'\u554f\u984c', u'\u300d', u'_', u'\u56e0\u70ba', u'\u9019\u727d\u6d89', u'\u6574\u9ad4', u'\u80fd\u6e90', u'\u653f\u7b56', u'\u7684', u'\u898f\u5283', u'_', u'\u800c', u'\u53f0\u7063', u'\u5fc5\u9808', u'\u5236\u5b9a', u'\u4e00\u5957', u'\u5408\u7406', u'\u4e14\u6c38\u7e8c', u'\u7684', u'\u80fd\u6e90', u'\u653f\u7b56', u'_', u'\u300c', u'\u76f8\u4fe1', u'\u5275\u8fa6', u'\u4eba', u'\u8ddf', u'\u6211', u'\u90fd', u'\u6709', u'\u4e00\u6a23', u'\u7684', u'\u5171\u8b58', u'\u300d', u'_', u'\u8521', u'\u4e5f', u'\u6f84\u6e05', u'_', u'\u904e\u53bb', u'\u5979', u'\u7684', u'\u96fb\u50f9', u'\u653f\u7b56', u'_', u'\u88ab', u'\u7c21', u'\u5316\u6210', u'10', u'\u5e74', u'\u4e0d\u6f32', u'\u96fb\u50f9', u'\u7684', u'\u65e2\u5b9a', u'\u5370\u8c61', u'_', u'\u4f46\u6c11', u'\u9032\u9ee8', u'\u5176\u5be6', u'\u662f', u'\u6709', u'\u5b8c\u6574', u'\u898f\u5283', u'\u8207', u'\u601d\u8003', u'\u7684', u'_', u'\u5979', u'\u7684', u'\u96fb\u50f9', u'\u4e3b\u5f35', u'_', u'\u662f', u'\u4ee5', u'\u57fa\u672c', u'\u5ea6\u6578', u'\u4e0d\u6f32', u'\u50f9\u70ba', u'\u539f\u5247', u'_', u'\u6e1b\u5c11', u'\u96fb\u50f9', u'\u6ce2\u52d5', u'_', u'\u56e0\u70ba', u'\u96fb\u50f9', u'\u4e00\u76f4', u'\u662f', u'\u53f0\u7063', u'\u751f\u7522', u'\u3001', u'\u751f\u6d3b', u'\u6210\u672c', u'\u7684', u'\u6700', u'\u91cd\u8981', u'\u7684', u'\u4e8b\u60c5', u'\u4e4b\u4e00', u'\uff1b', u'\u672a\u4f86', u'\u5373\u4fbf', u'\u8981', u'\u8abf\u6574', u'_', u'\u4e5f', u'\u6703', u'\u8003\u91cf', u'\u7269\u50f9', u'\u8ddf', u'\u4e00\u822c', u'\u4eba', u'\u8ca0\u64d4', u'\u80fd\u529b', u'_', u'\u5728', u'\u6c11\u751f', u'\u7528', u'\u96fb\u5c0f\u6236', u'\u90e8\u5206', u'_', u'\u300c', u'\u73fe\u968e\u6bb5', u'\u6c92\u6709', u'\u8abf\u6f32', u'\u8a08\u756b', u'\u300d', u'_', u'\u4f46\u7528', u'\u96fb\u5927\u6236\u5247', u'\u662f', u'\u91cd\u9ede', u'_', u'\u672a\u4f86', u'\u5c07\u9f13\u52f5', u'\u5927\u6236', u'\u914d\u5408', u'\u7bc0\u96fb', u'\u63aa\u65bd', u'_', u'\u4ee5\u53ca', u'\u4f9d\u7167', u'\u300c', u'\u6642\u9593', u'\u96fb\u50f9', u'\u300d', u'\u548c', u'\u300c', u'\u9700\u91cf', u'\u7af6\u50f9', u'\u300d', u'\u4f86', u'\u8abf\u5ea6', u'\u96fb\u529b', u'_', u'\u4f01\u696d', u'\u653f\u5e9c', u'3', u'\u7ba1\u9f4a', u'\u4e0b', u'\u8521', u'\u82f1\u6587', u'\u4e5f', u'\u8aaa', u'_', u'\u5979', u'\u4e4b\u6240\u4ee5', u'\u6709\u628a\u63e1', u'\u5730\u8aaa', u'\u300c', u'10', u'\u5e74', u'\u5167', u'\u96fb\u50f9', u'\u4e0d\u6703', u'\u5927\u5e45', u'\u4e0a\u6f32', u'\u300d', u'_', u'\u539f\u56e0', u'\u6709', u'3', u'_', u'\u5176\u4e00', u'_', u'\u53ea\u8981', u'\u843d\u5be6', u'\u7bc0\u80fd', u'_', u'\u96fb\u529b', u'\u4f7f\u7528', u'\u6548\u7387', u'\u589e\u52a0', u'_', u'\u5c31', u'\u4e0d', u'\u9700\u8981', u'\u591a\u7e73', u'\u96fb\u8cbb', u'\uff1b', u'\u5176\u4e8c', u'_', u'\u7da0\u80fd', u'\u73fe\u968e\u6bb5', u'\u50f9\u683c', u'\u96d6\u9ad8', u'_', u'\u4f46', u'\u79d1\u6280', u'\u9032\u6b65', u'\u3001', u'\u898f\u6a21', u'\u64f4\u5927', u'\u5f8c', u'_', u'\u6210\u672c', u'\u81ea\u7136', u'\u4e0b\u964d', u'\uff1b', u'\u5176\u4e09', u'_', u'\u65b0\u653f\u5e9c', u'\u5c07\u6703', u'\u4fee\u6539', u'\u96fb\u696d\u6cd5', u'_', u'\u63a8\u52d5\u5ee0', u'\u3001', u'\u7db2\u5206', u'\u96e2', u'_', u'\u63d0\u5347', u'\u53f0\u96fb\u7d93\u71df', u'\u6548\u7387', u'_', u'\u8b93', u'\u96fb\u50f9', u'\u66f4', u'\u900f\u660e', u'\u3001', u'\u66f4\u70ba', u'\u5408\u7406', u'_', u'\u5979', u'\u4e5f', u'\u5f37\u8abf', u'_', u'\u65b0\u653f\u5e9c', u'\u6703\u6301\u7e8c', u'\u8ddf', u'\u7522\u696d\u754c', u'\u6e9d\u901a', u'_', u'\u8996\u7e3d\u9ad4', u'\u7d93\u6fdf', u'\u60c5\u6cc1', u'_', u'\u505a', u'\u9069\u5ea6', u'\u7684', u'\u6aa2\u8a0e', u'_', u'\u4e26\u4e14', u'\u4f5c\u8abf', u'\u6574\u53ca', u'\u6539\u5584', u'_', u'\u8521', u'\u82f1\u6587', u'\u4e26', u'\u76db', u'\u8b9a', u'\u53f0', u'\u9054', u'\u96fb', u'_', u'\u8868\u793a', u'\u5979', u'\u7684', u'\u7522\u696d', u'\u4e4b', u'\u65c5\u5728', u'\u300c', u'\u65b0\u80fd\u6e90', u'\u300d', u'\u3001', u'\u300c', u'\u74b0\u4fdd\u7bc0', u'\u80fd\u7522\u696d', u'\u300d', u'\u9019', u'\u4e00\u7ad9', u'\u7279\u5225', u'\u4f86', u'\u5230', u'\u53f0\u9054\u96fb', u'_', u'\u662f', u'\u56e0\u70ba', u'\u300c', u'\u7bc0\u80fd', u'\u300d', u'\u662f', u'\u6700\u5feb', u'\u89e3\u6c7a', u'\u80fd\u6e90', u'\u4f9b\u7d66', u'\u554f\u984c', u'\u7684', u'\u65b9\u6cd5', u'_', u'\u800c\u53f0', u'\u9054\u96fb', u'\u53ef\u4ee5', u'\u8aaa', u'\u662f', u'\u7bc0\u80fd', u'\u8207', u'\u518d\u751f\u80fd\u6e90', u'\u76f8\u95dc', u'\u7522\u54c1', u'\u6700\u9f4a\u5168', u'\u7684', u'\u516c\u53f8', u'_', u'\u5979', u'\u4e5f', u'\u958b', u'\u73a9\u7b11', u'\u8aaa', u'_', u'\u300c', u'\u6211', u'\u7684', u'\u8b1b\u7a3f\u88e1', u'\u539f\u672c', u'\u662f', u'\u5beb', u'\u300e', u'\u4e4b\u4e00', u'\u300f', u'\u5566', u'_', u'\u6211', u'\u53bb\u6389', u'\u4e86', u'\uff01', u'\u300d', u'\u8b93', u'\u5728', u'\u5834\u53f0', u'\u9054\u96fb', u'\u4e3b\u7ba1', u'\u5011', u'\u4e00\u9663', u'\u9f13\u638c', u'\u7206\u7b11', u'_', u'\u7701\u96fb\u6a19', u'\u7aff', u'\u53f0\u9054\u96fb', u'\u8521', u'\u82f1\u6587', u'\u8aaa', u'_', u'\u53f0\u7063\u8981', u'\u5efa\u7acb', u'\u4e00\u500b', u'\u66f4', u'\u4e7e\u6de8', u'\u3001', u'\u66f4', u'\u6709\u6548\u7387', u'\u7684', u'\u80fd\u6e90', u'\u7cfb\u7d71', u'_', u'\u4e00\u5b9a', u'\u8981', u'\u501f\u52a9', u'\u53f0\u9054\u96fb', u'\u7684', u'\u767c\u5c55', u'\u7d93\u9a57', u'_', u'\u4e5f', u'\u671f\u5f85', u'\u53f0\u9054', u'\u96fb\u80fd\u5920', u'\u6574\u5408', u'\u4e0d\u540c', u'\u4e8b\u696d', u'\u7fa4\u7522\u54c1', u'_', u'\u898f\u5283', u'\u66f4\u597d', u'\u7684', u'\u89e3\u6c7a', u'\u65b9\u6848', u'\u4f86\u9762', u'\u5c0d', u'\u5168\u7403', u'\u7684', u'\u5e02\u5834', u'_', u'\u912d\u5d07\u83ef', u'\u4e5f', u'\u7576\u5834', u'\u5141\u8afe', u'_', u'\u672a\u4f86', u'\u4e00\u5b9a', u'\u6703', u'\u548c', u'\u65b0\u653f\u5e9c', u'\u5408\u4f5c', u'_', u'\u63a8\u52d5', u'\u7da0\u80fd', u'\u7522\u696d', u'_', u'(', u'\u4e2d\u570b', u'\u6642\u5831', u')']
SparseVector= {1507: 1.8326, 2751: 0.9163, 7410: 1.8326, 8321: 1.8326, 8414: 1.562, 11003: 0.9163, 15780: 0.9163, 28789: 1.5325, 29394: 0.9163, 32026: 0.9163, 32838: 1.8326, 46513: 1.0217, 64972: 0.9163, 72241: 0.2231, 79280: 1.8326, 81546: 0.5108, 86385: 0.9163, 95123: 0.9163, 101484: 2.7489, 111822: 1.8326, 116717: 0.5108, 130072: 2.7489, 134507: 0.9163, 138090: 0.5108, 144437: 0.9163, 153579: 0.9163, 160821: 0.9163, 167015: 0.9163, 174638: 0.9163, 184164: 0.9163, 188559: 0.9163, 190115: 0.9163, 206200: 0.9163, 212033: 0.9163, 215903: 0.5108, 220672: 0.5108, 226134: 0.9163, 226452: 0.9163, 228786: 0.9163, 229131: 0.9163, 242538: 0.9163, 242670: 0.5108, 244022: 0.9163, 245130: 0.9163, 247556: 0.9163, 251305: 0.9163, 255634: 1.8326, 255938: 1.8326, 256378: 2.7489, 261516: 0.9163, 278149: 1.8326, 278666: 0.9163, 284833: 0.9163, 285756: 0.9163, 288250: 0.9163, 295871: 2.7489, 297509: 0.9163, 300473: 0.9163, 303667: 0.9163, 308286: 0.9163, 309129: 2.7489, 313992: 2.0433, 316257: 0.9163, 316824: 0.9163, 318756: 0.9163, 320345: 0.9163, 323160: 0.9163, 329089: 0.9163, 331940: 0.9163, 338215: 0.9163, 340439: 0.9163, 340698: 0.9163, 347492: 0.9163, 362557: 0.9163, 366194: 0.9163, 367114: 0.5108, 369165: 2.4546, 378864: 0.9163, 386631: 0.5108, 392309: 0.5108, 398358: 0.9163, 401988: 0.9163, 403560: 0.9163, 405637: 4.686, 407314: 2.0433, 410979: 3.6652, 413727: 3.6652, 414791: 0.9163, 415046: 0.9163, 416656: 3.6652, 418435: 0.9163, 421404: 1.0217, 421461: 0.9163, 422974: 0.9163, 425905: 1.0217, 426710: 0.9163, 426892: 2.4546, 428181: 0.9163, 428924: 0.2231, 430099: 0.9163, 432070: 0.9163, 432678: 1.8326, 433328: 0.5108, 434054: 2.7489, 434211: 0.9163, 438848: 0.9163, 439122: 0.9163, 442178: 0.9163, 447276: 1.8326, 448458: 1.8326, 451694: 0.9163, 461064: 0.9163, 462810: 0.9163, 465339: 1.5325, 467766: 0.9163, 468094: 0.9163, 472058: 0.9163, 479024: 0.9163, 479604: 2.7489, 481016: 0.9163, 488734: 2.7489, 490427: 0.9163, 492433: 0.9163, 503229: 0.9163, 505252: 0.9163, 506212: 0.5108, 514234: 0.9163, 517391: 0.5108, 522507: 0.9163, 527965: 1.8326, 528006: 0.9163, 529289: 0.9163, 535273: 0.9163, 539326: 0.9163, 547839: 0.9163, 551281: 0.9163, 562079: 0.9163, 564476: 0.9163, 565806: 1.8326, 566482: 2.7489, 568616: 0.9163, 575118: 0.5108, 575637: 0.9163, 581210: 0.9163, 585612: 1.0217, 590372: 0.5108, 592490: 1.8326, 593447: 0.9163, 594122: 11.9118, 594304: 0.0, 594502: 0.9163, 594600: 1.8326, 623654: 0.9163, 626363: 0.9163, 630474: 1.8326, 631031: 0.9163, 633377: 0.5108, 633635: 1.0217, 634034: 0.6694, 638321: 1.8326, 640333: 0.9163, 642184: 0.9163, 642827: 0.9163, 652505: 0.9163, 653228: 0.9163, 659169: 0.9163, 662937: 0.9163, 666368: 0.9163, 676239: 0.9163, 679421: 0.9163, 682118: 0.8926, 686604: 0.9163, 691331: 0.9163, 693453: 0.9163, 693614: 0.5108, 700966: 0.9163, 702126: 1.8326, 702284: 1.8326, 703374: 0.9163, 703493: 0.9163, 707239: 0.9163, 707583: 1.8326, 708440: 0.9163, 714497: 0.5108, 717908: 0.9163, 719411: 0.9163, 722308: 0.9163, 724179: 0.9163, 729788: 0.9163, 747920: 0.9163, 749278: 0.0, 751304: 0.4463, 754981: 4.5815, 759077: 0.9163, 759543: 0.9163, 761443: 0.5108, 768302: 0.9163, 768937: 2.0433, 769009: 0.9163, 777008: 0.9163, 786349: 0.9163, 786959: 0.9163, 788109: 0.4463, 790789: 1.8326, 792667: 0.9163, 795756: 0.9163, 795872: 0.9163, 795881: 0.9163, 795914: 0.9163, 798581: 0.9163, 798982: 0.9163, 802034: 0.9163, 804503: 0.9163, 806522: 0.9163, 808096: 6.414, 809294: 0.2231, 814833: 0.5108, 817809: 1.8326, 819133: 0.5108, 820024: 0.9163, 821493: 0.6694, 822669: 0.2231, 826792: 0.9163, 833940: 0.9163, 836013: 0.9163, 837511: 0.2231, 837929: 0.9163, 840422: 1.8326, 842455: 2.7489, 842919: 1.8326, 844265: 0.5108, 845088: 0.5108, 849130: 0.9163, 854923: 1.8326, 856405: 0.9163, 856589: 0.5108, 861905: 0.9163, 867369: 0.2231, 869296: 0.9163, 885634: 1.0217, 890715: 0.9163, 891817: 0.9163, 894568: 0.9163, 895974: 0.5108, 903829: 0.9163, 911402: 0.9163, 915354: 0.6694, 916669: 0.9163, 920706: 0.9163, 920824: 5.4977, 920868: 0.9163, 924921: 1.0217, 926281: 0.9163, 928923: 0.9163, 934062: 1.7851, 935190: 4.5815, 935559: 1.0217, 941480: 0.2231, 943673: 0.9163, 943703: 0.9163, 945791: 0.9163, 956065: 1.8326, 956432: 0.9163, 957308: 0.5108, 959722: 0.9163, 974526: 0.9163, 975894: 0.9163, 976327: 0.9163, 980196: 0.9163, 983073: 0.5108, 987355: 0.9163, 990454: 0.9163, 992094: 2.7489, 994720: 0.9163, 995736: 1.0217, 996181: 0.9163, 997897: 0.9163, 1002337: 1.5325, 1003220: 0.9163, 1007232: 0.5108, 1008691: 0.9163, 1012696: 0.9163, 1019295: 0.5108, 1020726: 0.9163, 1025488: 0.9163, 1025639: 0.9163, 1026513: 0.9163, 1028435: 1.8326, 1029547: 2.0433, 1029927: 0.9163, 1033388: 0.5108, 1034799: 0.9163, 1042432: 1.8326, 1042478: 0.9163, 1043241: 0.4463, 1044929: 0.9163, 1047597: 0.9163, 1047649: 0.9163}

#for i in keywords:
    #print (i)
#ii=[283648, 892929, 478723, 578566, 367114, 947211, 369165, 33806, 961039, 240146, 990467, 837511, 934062, 112662, 970264, 600601, 51226, 179739, 23580, 58910, 852702, 274465, 34338, 1025115, 814342, 672177, 781864, 819133, 405034, 248363, 1007148, 473646, 72241, 139826, 708149, 1022112, 993766, 344120, 782260, 532490, 895038, 871605, 721473, 784962, 772675, 642631, 1047112, 206409, 35914, 1041847, 454732, 856589, 153168, 525752, 984148, 206421, 324694, 440408, 64601, 698971, 749278, 688734, 725601, 542819, 739941, 986214, 821868, 630546, 166511, 977622, 897896, 633377, 870590, 73847, 254585, 6780, 23678, 251520, 457345, 1001835, 405637, 27782, 313992, 194697, 270474, 785035, 788109, 575118, 854513, 990745, 399514, 743583, 698528, 608929, 322210, 727067, 933433, 172710, 459432, 274601, 673966, 120495, 1032575, 751304, 634034, 587443, 126494, 930486, 777673, 892073, 5818, 938171, 861886, 204991, 1014976, 727238, 9415, 148680, 1029547, 962639, 48331, 908492, 957257, 52430, 54479, 527056, 406230, 533113, 436952, 777945, 263631, 8414, 426208, 864993, 700643, 9957, 629990, 888929, 1002620, 72939, 47852, 407277, 230129, 815346, 608688, 450804, 821493, 788215, 924377, 1046280, 334075, 7932, 786173, 272000, 485122, 515331, 583382, 187142, 692487, 255240, 401674, 517391, 407314, 511764, 599831, 937515, 116507, 848977, 845088, 271649, 633635, 407844, 682118, 522022, 844265, 1043241, 149802, 589703, 413484, 845618, 572718, 483976, 33586, 559923, 182494, 856375, 368952, 886059, 440635, 923453, 365376, 120641, 612162, 180036, 531269, 462150, 190280, 533388, 809294, 353615, 801507, 168276, 719190, 405335, 206223, 990563, 506212, 894438, 522088, 138090, 808811, 519533, 693614, 535414, 72055, 325496, 428924, 559082, 904575, 594304, 110977, 326530, 755075, 591750, 308033, 893164, 891275, 426892, 822669, 769345, 9615, 864145, 134547, 924746, 226709, 408985, 915354, 784796, 1042333, 872993, 115622, 486897, 250792, 741959, 503212, 1013679, 593328, 425905, 15795, 269748, 1038774, 746935, 30136, 465339, 190026, 524736, 864193, 999942, 496068, 646732, 976631, 395725, 242127, 875596, 977344, 419799, 943597, 190939, 996517, 810572, 728315, 583846, 290790, 615399, 164604, 354794, 783271, 525293, 186865, 928527, 347638, 255991, 1007232, 815101, 934398]   
#print ((SparseVector))    
#print (list(SparseVector)) 
#for dic in SparseVector:
    #print (SparseVector[dic])

In [57]:
new=[]
for keyword,i in zip(keywords,SparseVector):
    #print (keyword,SparseVector[i])
    new.append((keyword,SparseVector[i]))
#print (new)
new1=sorted(new, key = lambda x : x[1],reverse=True)
#print (new1)

#for dic in new1:
    #print (dic[0],dic[1])
    
for ele in new1:
    fil = re.sub("[\s+\.\!\/_,$%^*「」(+\"\']+|[+——！，。？、~@#￥%……&*（）]+".decode("utf-8"), "".decode("utf-8"),ele[0])
    if fil =="":
        pass
    else:
        print (ele[0],ele[1])   # show keywords and count
print ("---------next one----------")

鎖定 6.414
來 5.4977
政策 4.686
這牽涉 4.5815
她 4.5815
選前 3.6652
因為 3.6652
負擔 3.6652
由 2.7489
科技 2.7489
的 2.7489
制定 2.7489
電價 2.7489
減少 2.7489
生活 2.7489
重要 2.7489
調整 2.7489
也 2.7489
以及 2.7489
辦人 2.4546
電價 2.4546
小戶 2.0433
昨天 2.0433
也 2.0433
地說 2.0433
英文 1.8326
問題 1.8326
10 1.8326
落實節 1.8326
廠 1.8326
離 1.8326
選後展 1.8326
開產業 1.8326
參訪 1.8326
舉行 1.8326
蔡在致 1.8326
鄭曾 1.8326
知道 1.8326
規劃 1.8326
且永續 1.8326
的 1.8326
過去 1.8326
的 1.8326
主張 1.8326
基本 1.8326
考量 1.8326
重點 1.8326
電價 1.8326
和 1.8326
3 1.8326
年 1.8326
談 1.7851
因為 1.562
以及 1.5325
措施 1.5325
於 1.0217
隔 1.0217
相信 1.0217
簡 1.0217
即便 1.0217
英文 1.0217
蔡 0.9163
昨日 0.9163
南下 0.9163
台達 0.9163
電創 0.9163
鄭崇華 0.9163
等 0.9163
人 0.9163
對 0.9163
曾 0.9163
電價 0.9163
昨天 0.9163
不僅 0.9163
澄清 0.9163
電價 0.9163
與 0.9163
能源 0.9163
政策 0.9163
並強 0.9163
調 0.9163
之所以 0.9163
說出 0.9163
電價 0.9163
大幅 0.9163
的 0.9163
原因 0.9163
有 0.9163
3 0.9163
包括 0.9163
能 0.9163
讓 0.9163
成本 0.9163
下降 0.9163
綠能 0.9163
規模 0.9163
可 0.9163
降低成本 0.9163
修改 0.9163
《 0.9163
電業法 0.9163
》 0.9163
網分 0